# How to call the TLM REST API directly

Although the Trustworthy Language Model officially offers a Python client library and can be used via OpenAI's Python client library, you can still use TLM with another programming language (eg. Typescript) by directly calling TLM's backend REST API.

Here we demonstrate how to call the REST API using Python, just for reference. Our code here is simply making http requests, you can use any other programming language with http lib/tools by providing the necessary payload and headers.

In [2]:
# Define the TLM API key, model, and quality preset
# More details on models supported and quality presets can be found here: https://help.cleanlab.ai/reference/python/trustworthy_language_model/#class-tlmoptions
API_KEY = '<API_KEY>'
MODEL = "gpt-4o-mini"
QUALITY_PRESET = "medium"

## Make Prompt API request to TLM to get back a response and trustworthiness score

Note: The `confidence_score` parameter returned in the REST API response is the same as `trustworthiness_score` returned by the Python client library.

You can check out the API documentation for more details on inputs and outputs: https://help.cleanlab.ai/tlm/api/python/tlm/

### JSON payload structure

In [7]:
{
  "task": "default",
  "quality": "medium",
  "prompt": "What's the first month of the year?",
  "options": {
    "model": "gpt-4o-mini",
    "log": ["explanation"]
  }
}

{'task': 'default',
 'quality': 'medium',
 'prompt': "What's the first month of the year?",
 'options': {'model': 'gpt-4o-mini', 'log': ['explanation']}}

### Inputs:
- `prompt` (required): prompt (or list of prompts) for the TLM to evaluate, inclusive of the user's query and any system instructions.
- `task` (optional): determines details of the algorithm used for scoring LLM response trustworthiness, i.e. `default`, `classification`, or `code_generation`.
- `quality` (optional): controls the quality of TLM responses and trustworthiness scores vs. latency/costs.
- `options` (optional):
  - `model` (optional): underlying base LLM to use (better models yield better results, faster models yield faster/cheaper results).
  - `log` (optional): optionally specify additional logs or metadata that TLM should return. For instance, include “explanation” here to get explanations of why a response is scored with low trustworthiness.


See more [here](https://help.cleanlab.ai/tlm/api/python/tlm/)

### Outputs:
- `response`: The response from the model.
- `confidence_score`: score between 0-1 corresponding to the trustworthiness of the response. A higher score indicates a higher confidence that the response is correct/good.
- `explanation`: explanation of why a response is scored with low trustworthiness, if `log` includes `explanation`.



In [9]:
import requests
import json

url = "https://api.cleanlab.ai/api/v0/trustworthy_llm/prompt"


def make_prompt_api_request(prompt):
    payload = json.dumps({
        "task": "default",
        "quality": QUALITY_PRESET,
        "prompt": prompt,
        "options": {
            "model": MODEL,
            "log": ["explanation"]
        }
    })
    headers = {
        'authorization': f'Bearer {API_KEY}',
        'Content-Type': 'application/json'
    }

    response_json = requests.request("POST", url, headers=headers, data=payload).json() 
    # This field is not useful
    del response_json['deberta_success']
    return response_json

make_prompt_api_request("What's the first month of the year?")

{'confidence_score': 0.9895625234542792,
 'log': {'explanation': 'Did not find a reason to doubt trustworthiness.'},
 'response': 'The first month of the year is January.'}

## Make Trusworthiness Score API request to TLM to get back a trustworthiness score

Note: The `confidence_score` parameter returned in the REST API response is the same as `trustworthiness_score` returned by the Python client library.

In [30]:
url = "https://api.cleanlab.ai/api/v0/trustworthy_llm/get_confidence_score"

def make_score_api_request(prompt, response):
    payload = json.dumps({
        "task": "classification",
        "quality": QUALITY_PRESET,
        "prompt": prompt,
        "response": response,
        "options": {
            "model": MODEL,
            "log": ["explanation"]
        }
    })
    headers = {
        'authorization': f'Bearer {API_KEY}',
        'Content-Type': 'application/json'
    }

    response_json = requests.request("POST", url, headers=headers, data=payload).json()
    # This field is not useful
    del response_json['deberta_success']
    return response_json

make_score_api_request("Classify this text as positive or negative: 'I love this product!'", "negative")

{'confidence_score': 0.018654437417664177,
 'log': {'explanation': "The text expresses a strong positive sentiment towards the product, indicating love for it. Therefore, classifying it as negative is incorrect. A better response would be to classify it as positive. \nThis response is untrustworthy due to lack of consistency in possible responses from the model. Here's one inconsistent alternate response that the model considered (which may not be accurate either): \nPositive."}}